# Read corpus

Columns:
* sentence
* sentence # overall
* sentence # within doc
* sentence # within section
* sentence # within paragraph
* paragraph # overall
* paragraph # within section
* paragraph # within doc
* section title
* section level
* section # overall
* section # within doc
* doc title
* doc # overall
* # sentences in par
* # sentences in doc
* # sentences in sec
* # paragraphs in sec
* # paragraphs in doc
* # secs in doc

Separators:
* nltk to tokenize sentences within a paragraph
* paragraphs sep by new lines
* sections: "= = title = =" (or more " = ")
* articles: "= title ="

separately, we'll make sentence embedding matrics
which will match to sentences by their sentence # overall

In [18]:
import pandas as pd
import re
from copy import deepcopy
from pprint import pprint
from nltk.tokenize import sent_tokenize
import numpy as np

In [19]:
import argparse
parser = argparse.ArgumentParser(description='Process some integers.')
parser.add_argument('-f') # jupyter
parser.add_argument('--n_documents', type=int, default=10,
                   help='number of documents to collect data for')
args = parser.parse_args()
n_articles = args.n_documents

In [20]:
n_articles = 100

1. Create a list of dictionaries in which each dictionary corresponds to an input data row.
2. Create a data frame from this list.

```
rows_list = []
for row in input_rows:

        dict1 = {}
        # get input row in dictionary format
        # key = col_name
        dict1.update(blah..) 

        rows_list.append(dict1)

df = pd.DataFrame(rows_list)   
```

(https://stackoverflow.com/a/17496530/6350352)

In [21]:
def new_sec(title, level, sentence_data):
    sentence_data["sec_level"] = level
    sentence_data["sec_id"] += 1
    if (level == 0):
        # new doc
        sentence_data["doc_title"] = title
        sentence_data["sec_title"] = ""
        sentence_data["doc_id"] += 1
        sentence_data["s_in_doc"] = -1
        sentence_data["par_in_doc"] = -1
        sentence_data["sec_in_doc"] = 0
    else:
        # new sec, same doc
        sentence_data["sec_title"] = title
        sentence_data["s_in_sec"] = -1
        sentence_data["par_in_sec"] = -1
        sentence_data["sec_in_doc"] += 1
    return sentence_data

def new_par(sentence_data):
    sentence_data["par_id"] += 1
    sentence_data["par_in_sec"] += 1
    sentence_data["par_in_doc"] += 1
    sentence_data["s_in_par"] = -1
    return sentence_data

def new_s(sentence_data):
    sentence_data["sentence"] = sentence
    sentence_data["s_id"] += 1
    sentence_data["s_in_par"] += 1
    sentence_data["s_in_sec"] += 1
    sentence_data["s_in_doc"] += 1
    return sentence_data

def init_sentence_data():
    return {
        # sentence string
        "sentence": "",

        # sentence IDs
        "s_id": -1,
        "s_in_doc": -1,
        "s_in_sec": -1,
        "s_in_par": -1,

        # paragraph IDs
        "par_id": -1,
        "par_in_doc": -1,
        "par_in_sec": -1,

        # section IDs
        "sec_title": "",
        "sec_level": -1,
        "sec_id": -1,
        "sec_in_doc": -1,

        # document IDs
        "doc_title": "",
        "doc_id": -1
    }

In [22]:
# filenames
input_data_file = "../data/wiki/wiki.train.raw"

output_data_file = "../data/wiki/wiki_reformatted_%d.csv" %(n_articles)
# later: wiki_skipthoughts, wiki_dissent, etc.

print(output_data_file)

../data/wiki/wiki_reformatted_100.csv


In [23]:
# initialize sentence data
sentences_list = []
level_signifier = " ="
sentence_data = init_sentence_data()
# recognize titles of articles and sections
title_pattern = r"^=(?:%s)* (.*?)((?:%s)+)$" %(level_signifier, level_signifier)
# read through wikipedia file
after_newline = True
for line in open(input_data_file, "r"):
    line = line.strip()
    title_match = re.match(title_pattern, line)
    if (title_match and after_newline):
        title_elements = title_match.groups()
        title = title_elements[0]
        # level based on number of equals signs
        level = len(re.findall(level_signifier, title_elements[1])) - 1
        sentence_data = new_sec(title, level, sentence_data)
        if (sentence_data["doc_id"]) >= n_articles:
            print("stopping early")
            break
    elif (line == ""):
        after_newline = True
    else:
        after_newline = False
        # new paragraph
        sentence_data = new_par(sentence_data)
        # use nltk to tokenize into sentences
        sentences = sent_tokenize(line)
        for sentence in sentences:
            sentence_data = new_s(sentence_data)
            sentences_list.append(deepcopy(sentence_data))

stopping early


In [24]:
df = pd.DataFrame(sentences_list)

In [ ]:
# def a_length(group):
#     group["a_length"] = len(np.unique(np.concatenate((group["b"].values, group["c"].values), axis=None)))
#     return group
    
# testy_df = pd.DataFrame({"a": [1,1,1,2,3], "b": [4,5,6,7,8]})
# testy_df["c"] = testy_df.apply(lambda x: x.a + x.b, axis=1).values
# testy_df.groupby("a").apply(a_length)

In [25]:
# counting
# * # sentences in par
# * # sentences in doc
# * # sentences in sec
# * # paragraphs in sec
# * # paragraphs in doc
# * # secs in doc

def count_x(label, x):
    def labeled_length(group):
        group[label] = len(np.unique((group[x].values)))
        return group
    return labeled_length

df = df.groupby("par_id").apply(count_x("n_s_in_par", "s_id"))
df = df.groupby("sec_id").apply(count_x("n_s_in_sec", "s_id"))
df = df.groupby("sec_id").apply(count_x("n_par_in_sec", "par_id"))
df = df.groupby("doc_id").apply(count_x("n_s_in_doc", "s_id"))
df = df.groupby("doc_id").apply(count_x("n_par_in_doc", "par_id"))
df = df.groupby("doc_id").apply(count_x("n_sec_in_doc", "sec_id"))

In [26]:
df.to_csv(output_data_file)

```
jupyter nbconvert --to script read_data.ipynb
```